In [22]:
import pandas as pd

In [23]:
wsb = pd.read_csv("data/wsb.csv")
wsb

,register_index,post_id,comment_id,author,datetime,title,url,score,comments,text,author_post_karma,tag
0,14b78hkjoe86nf,14b78hk,joe86nf,scott_jr,2023-06-16 20:36:55,NaN,NaN,1.0,NaN,Watch til 1 10,32102.0,Meme
1,14b71m2post,14b71m2,NaN,merakibret,2023-06-16 20:24:01,I had my first ever big success with options t...,https://www.reddit.com/r/wallstreetbets/commen...,8.0,6.0,Entered an Iron Condor on ADBE yesterday at 45...,343.0,Gain
2,14b71m2joe6du9,14b71m2,joe6du9,VisualMod,2023-06-16 20:24:07,NaN,NaN,1.0,NaN,User Report Tota...,725083.0,Gain
3,14b71m2joe6een,14b71m2,joe6een,VisualMod,2023-06-16 20:24:13,NaN,NaN,2.0,NaN,That was a very wise move,725083.0,Gain
4,14b71m2joe7yy4,14b71m2,joe7yy4,DreamcatcherEgg,2023-06-16 20:35:23,NaN,NaN,2.0,NaN,All you have to do is repeat this same winning...,6088.0,Gain
...,...,...,...,...,...,...,...,...,...,...,...,...
3033535,1j96owemhd8ajs,1j96owe,mhd8ajs,jarail,2025-03-12 11:25:28,NaN,NaN,4.0,NaN,Hopefully he made the whole story up,NaN,Discussion
3033536,1j96owemhcapeb,1j96owe,mhcapeb,South_Age974,2025-03-12 05:46:48,NaN,NaN,2.0,NaN,holy f,NaN,Discussion
3033537,1j96owemhjt5np,1j96owe,mhjt5np,PickinLosers,2025-03-13 11:49:05,NaN,NaN,2.0,NaN,I like to call them PDFs public domain fries,NaN,Discussion
3033538,1j96owemhex8ls,1j96owe,mhex8ls,The_Whackest,2025-03-12 17:04:15,NaN,NaN,1.0,NaN,Singsongy Some make you laugh and others ma...,NaN,Discussion


In [24]:
wsb.info()
wsb.head()
wsb.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033540 entries, 0 to 3033539
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   register_index     object 
 1   post_id            object 
 2   comment_id         object 
 3   author             object 
 4   datetime           object 
 5   title              object 
 6   url                object 
 7   score              float64
 8   comments           float64
 9   text               object 
 10  author_post_karma  float64
 11  tag                object 
dtypes: float64(3), object(9)
memory usage: 277.7+ MB


(3033540, 12)

In [25]:
# Clean and normalize dataset

# drop uneccessary columns
wsb = wsb.drop(columns=["register_index"])

# convert datetime column to datetime type and normalize to date only
wsb["datetime"] = pd.to_datetime(wsb["datetime"], errors="coerce").dt.normalize()

# convert numeric columns to int type and fill na with 0
numeric_cols = ["score", "comments", "author_post_karma"]
for col in numeric_cols:
    wsb[col] = pd.to_numeric(wsb[col], errors="coerce")
    wsb[col] = wsb[col].fillna(0).astype(int)

# clean and normalize tag column
wsb["tag"] = (
    wsb["tag"].fillna("unknown").str.strip().str.lower().str.replace(r"\s+", "_", regex=True)
)
wsb["tag"] = wsb["tag"].astype("category")

# identify post or comment
wsb["is_post"] = wsb["comment_id"].isna()

wsb.info()
wsb.head()
wsb.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033540 entries, 0 to 3033539
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   post_id            object        
 1   comment_id         object        
 2   author             object        
 3   datetime           datetime64[ns]
 4   title              object        
 5   url                object        
 6   score              int64         
 7   comments           int64         
 8   text               object        
 9   author_post_karma  int64         
 10  tag                category      
 11  is_post            bool          
dtypes: bool(1), category(1), datetime64[ns](1), int64(3), object(6)
memory usage: 237.2+ MB


(3033540, 12)

In [ ]:
# basic filtering / denoising

# remove obvious bot/moderator authors
bot_users = ["VisualMod", "AutoModerator"]
wsb = wsb[~wsb["author"].isin(bot_users)]


# drop rows with deleted/empty content
garbage_tokens = {"", "[deleted]", "[removed]"}
title_clean = wsb["title"].fillna("").str.strip()
text_clean = wsb["text"].fillna("").str.strip()

title_garbage = title_clean.isin(garbage_tokens)
text_garbage = text_clean.isin(garbage_tokens)

post_mask = wsb["is_post"]
comment_mask = ~wsb["is_post"]

drop_mask = (post_mask & title_garbage & text_garbage) | (comment_mask & text_garbage)
wsb = wsb[~drop_mask]

wsb.shape


(2956091, 12)

In [ ]:
# build a text field for nlp